## 数据再次处理
- 数据去重
- 时差计算
- 标签生成
- 再次分析

In [2]:
import pandas as pd
import logging
pd.set_option('display.max_columns', 1000)

In [13]:
data = pd.read_csv('data_train_comp.csv')

/home/wangh/softwares/miniconda3/envs/pytorch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
def drop_dups(data, keys):

    logging.info('删除数据中的重复条目')
    logging.info('主键：{0}'.format(keys))
    logging.info('去重前数据大小：{0}'.format(data.shape))
    data_out = data.drop_duplicates(
        subset=keys, keep='first', inplace=False)
    logging.info('去重后数据大小：{0}'.format(data_out.shape))

    return data_out
data.drop(labels=['manufacturer_add', 'vendor_add',
            'manufacturer_kernel', 'vendor_kernel'], axis=1, inplace=True)  # 删去冗余补全行
data = drop_dups(data, ['serial_number', 'collect_time','mca_id','row', 'col', 'rankid', 'memory']) # 最后去重

In [15]:
data.loc[:, 'collect_time'] = pd.to_datetime(data.collect_time)
data.loc[:, 'failure_time'] = pd.to_datetime(data.failure_time)
data.interval = data.failure_time - data.collect_time
# data.interval.fillna('99 days 00:00:00', inplace=True)

In [16]:
# 做label（二分+不确定）
data.loc[:,'label'] = data['interval'] < pd.Timedelta('7 days 00:00:00')
# 对不确定的区域（进行回归预测修正）

In [17]:
data.to_csv('data_train_final_.csv')

## 数据读取

In [3]:
data = pd.read_csv('data_train_final_.csv', index_col=0)

/home/wangh/softwares/miniconda3/envs/pytorch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/wangh/softwares/miniconda3/envs/pytorch/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [22]:
data.columns

Index(['serial_number', 'memory', 'rankid', 'bankid', 'row', 'col',
       'collect_time', 'e0', 'e1', 'e2', 'e3', 'e4', 'e5', 'e6', 'e7', 'e8',
       'e9', 'e10', 'e11', 'e12', 'e13', 'e14', 'e15', 'e16', 'e17', 'e18',
       'e19', 'e20', 'e21', 'e22', 'e23', 'mca_id', 'transaction',
       'manufacturer', 'vendor', 'failure_time', 'failure_tag', 'interval',
       'label'],
      dtype='object')

In [4]:
mca_id_map = {}
idx = 0
for item in data.mca_id.unique():
    mca_id_map[item] = idx
    idx += 1
data.loc[:, 'mca_id'] = data.mca_id.map(lambda x: mca_id_map[x])

In [5]:
data = data.drop(columns=['serial_number', 'collect_time','failure_time', 'failure_tag', 'interval'])

### 对error formula与error的共现性做count

In [45]:
data[data.label==True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 496874 entries, 25416747 to 27570778
Data columns (total 34 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   memory        473123 non-null  float64
 1   rankid        473123 non-null  float64
 2   bankid        473123 non-null  float64
 3   row           473123 non-null  float64
 4   col           473123 non-null  float64
 5   e0            36 non-null      float64
 6   e1            534 non-null     float64
 7   e2            108 non-null     float64
 8   e3            0 non-null       float64
 9   e4            0 non-null       float64
 10  e5            0 non-null       float64
 11  e6            0 non-null       float64
 12  e7            236 non-null     float64
 13  e8            23 non-null      float64
 14  e9            241 non-null     float64
 15  e10           223 non-null     float64
 16  e11           374 non-null     float64
 17  e12           7477 non-null    float64


In [6]:
import numpy as np 
import pandas as pd 
from pandas import Series, DataFrame 
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error 
import lightgbm as lgb 

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 

train, test = train_test_split(data, test_size = 0.3)
test, val = train_test_split(test, test_size = 0.5)

In [7]:
train_data = lgb.Dataset(train.iloc[:,:-1], label=train.iloc[:,-1])
validation_data = lgb.Dataset(val.iloc[:,:-1], label=val.iloc[:,-1])

## LGBM

In [8]:
import time
param = {'num_leaves':500, 'objective':'binary','max_depth':9,'learning_rate':.05,'max_bin':200, 'num_threads':36, 'device':'cpu', 'train_metric':True}
param['metric'] = ['auc', 'binary_logloss', 'binary_error']
# param:boosting, default=gbdt, type=enum, options=gbdt, rf, dart, goss, alias=boost, boosting_type
# 用 categorical_feature 指定分类特征 参考 Parameters 的参数 categorical_feature
# 首先需要转换为 int 类型，并且只支持非负数。 将其转换为连续范围更好。
# 使用 min_data_per_group, cat_smooth 去处理过拟合（当 #data 比较小，或者 #category 比较大）
# 对于具有高基数的分类特征(#category 比较大), 最好把它转化为数字特征。
# param['metric'] = ['auc', 'binary_logloss']

#Here we have set max_depth in xgb and LightGBM to 7 to have a fair comparison between the two.

#training our model using light gbm
num_round=500
start=time.time()
lgbm=lgb.train(param,train_data,num_round, valid_sets=validation_data)
stop=time.time()

#Execution time of the model
execution_time_lgbm = stop-start
print(execution_time_lgbm)

[LightGBM] [Info] Number of positive: 347777, number of negative: 33639980
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.968981 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 466
[LightGBM] [Info] Number of data points in the train set: 33987757, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.010232 -> initscore=-4.571909
[LightGBM] [Info] Start training from score -4.571909
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.97491	valid_0's binary_logloss: 0.039628	valid_0's binary_error: 0.00815176
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.986469	valid_0's binary_logloss: 0.035149	valid_0's binary_error: 0.00785834
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[45]	valid_0's auc: 0.997421	valid_0's binary_logloss: 0.0134782	valid_0's binary_error: 0.00461919
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[46]	valid_0's auc: 0.997453	valid_0's binary_logloss: 0.0133451	valid_0's binary_error: 0.00459407
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[47]	valid_0's auc: 0.997486	valid_0's binary_logloss: 0.0132205	valid_0's binary_error: 0.00455027
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[48]	valid_0's auc: 0.997516	valid_0's binary_logloss: 0.0131032	valid_0's binary_error: 0.00452871
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[49]	valid_0's auc: 0.997542	valid_0's binary_logloss: 0.0129779	valid_0's binary_error: 0.00449617
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	valid_0's auc: 0.99757	valid_0's binary_loglo

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[92]	valid_0's auc: 0.99842	valid_0's binary_logloss: 0.0099738	valid_0's binary_error: 0.0032743
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[93]	valid_0's auc: 0.998445	valid_0's binary_logloss: 0.00992209	valid_0's binary_error: 0.00326057
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[94]	valid_0's auc: 0.998461	valid_0's binary_logloss: 0.00986159	valid_0's binary_error: 0.0032294
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[95]	valid_0's auc: 0.998474	valid_0's binary_logloss: 0.00981794	valid_0's binary_error: 0.00321663
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[96]	valid_0's auc: 0.998492	valid_0's binary_logloss: 0.00975114	valid_0's binary_error: 0.00317997
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[97]	valid_0's auc: 0.998514	valid_0's binary_log

[138]	valid_0's auc: 0.998966	valid_0's binary_logloss: 0.00795214	valid_0's binary_error: 0.0023707
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[139]	valid_0's auc: 0.998986	valid_0's binary_logloss: 0.00788219	valid_0's binary_error: 0.00234804
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[140]	valid_0's auc: 0.99899	valid_0's binary_logloss: 0.00785969	valid_0's binary_error: 0.0023306
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[141]	valid_0's auc: 0.998996	valid_0's binary_logloss: 0.00783073	valid_0's binary_error: 0.00232484
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[142]	valid_0's auc: 0.998997	valid_0's binary_logloss: 0.00781634	valid_0's binary_error: 0.00231989
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[143]	valid_0's auc: 0.998999	valid_0's binary_logloss: 0.00780293	valid_0's binary_error: 0.00231221
[LightGBM] [Warn

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[185]	valid_0's auc: 0.999207	valid_0's binary_logloss: 0.00688636	valid_0's binary_error: 0.00201165
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[186]	valid_0's auc: 0.999215	valid_0's binary_logloss: 0.00684627	valid_0's binary_error: 0.00199805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[187]	valid_0's auc: 0.999223	valid_0's binary_logloss: 0.00680977	valid_0's binary_error: 0.00198734
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[188]	valid_0's auc: 0.999227	valid_0's binary_logloss: 0.00678571	valid_0's binary_error: 0.00198267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[189]	valid_0's auc: 0.999233	valid_0's binary_logloss: 0.00675777	valid_0's binary_error: 0.00198062
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[190]	valid_0's auc: 0.999236	valid_0's 

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[278]	valid_0's auc: 0.999472	valid_0's binary_logloss: 0.00533098	valid_0's binary_error: 0.00150238
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[279]	valid_0's auc: 0.999473	valid_0's binary_logloss: 0.0053192	valid_0's binary_error: 0.00149978
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[280]	valid_0's auc: 0.999476	valid_0's binary_logloss: 0.00530252	valid_0's binary_error: 0.00149236
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[281]	valid_0's auc: 0.999477	valid_0's binary_logloss: 0.0052927	valid_0's binary_error: 0.00148907
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[282]	valid_0's auc: 0.999479	valid_0's binary_logloss: 0.00528481	valid_0's binary_error: 0.00148838
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[283]	valid_0's auc: 0.999481	valid_0's bi

[324]	valid_0's auc: 0.99953	valid_0's binary_logloss: 0.00494704	valid_0's binary_error: 0.00141327
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[325]	valid_0's auc: 0.99953	valid_0's binary_logloss: 0.00494076	valid_0's binary_error: 0.00141121
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[326]	valid_0's auc: 0.999532	valid_0's binary_logloss: 0.00493265	valid_0's binary_error: 0.00140888
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[327]	valid_0's auc: 0.999532	valid_0's binary_logloss: 0.00493224	valid_0's binary_error: 0.00140874
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[328]	valid_0's auc: 0.999533	valid_0's binary_logloss: 0.00492706	valid_0's binary_error: 0.00140833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[329]	valid_0's auc: 0.999533	valid_0's binary_logloss: 0.00492642	valid_0's binary_error: 0.00140682
[LightGBM] [War

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[371]	valid_0's auc: 0.999571	valid_0's binary_logloss: 0.00465502	valid_0's binary_error: 0.00134256
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[372]	valid_0's auc: 0.999573	valid_0's binary_logloss: 0.00464381	valid_0's binary_error: 0.00133968
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[373]	valid_0's auc: 0.999574	valid_0's binary_logloss: 0.00463266	valid_0's binary_error: 0.00133707
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[374]	valid_0's auc: 0.999574	valid_0's binary_logloss: 0.00463213	valid_0's binary_error: 0.00133693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[375]	valid_0's auc: 0.999576	valid_0's binary_logloss: 0.00462495	valid_0's binary_error: 0.0013357
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[376]	valid_0's auc: 0.999576	valid_0's b

[417]	valid_0's auc: 0.999613	valid_0's binary_logloss: 0.00436402	valid_0's binary_error: 0.00129039
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[418]	valid_0's auc: 0.999613	valid_0's binary_logloss: 0.00435919	valid_0's binary_error: 0.00128819
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[419]	valid_0's auc: 0.999614	valid_0's binary_logloss: 0.00435461	valid_0's binary_error: 0.00128791
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[420]	valid_0's auc: 0.999614	valid_0's binary_logloss: 0.00435252	valid_0's binary_error: 0.00128805
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[421]	valid_0's auc: 0.999614	valid_0's binary_logloss: 0.00435207	valid_0's binary_error: 0.00128791
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[422]	valid_0's auc: 0.999614	valid_0's binary_logloss: 0.00435194	valid_0's binary_error: 0.00128791
[LightGBM] [W

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[464]	valid_0's auc: 0.99964	valid_0's binary_logloss: 0.00417115	valid_0's binary_error: 0.00124686
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[465]	valid_0's auc: 0.99964	valid_0's binary_logloss: 0.00416903	valid_0's binary_error: 0.00124672
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[466]	valid_0's auc: 0.99964	valid_0's binary_logloss: 0.00416898	valid_0's binary_error: 0.00124672
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[467]	valid_0's auc: 0.99964	valid_0's binary_logloss: 0.00416833	valid_0's binary_error: 0.00124672
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[468]	valid_0's auc: 0.99964	valid_0's binary_logloss: 0.00416405	valid_0's binary_error: 0.00124521
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[469]	valid_0's auc: 0.999641	valid_0's binar

In [ ]:
# train_data.save_binary('train.bin')
# # 保存数据加载更快
lgb.save_model('model.txt')

In [9]:
ypred=lgbm.predict(test.iloc[:,:-1])

In [15]:
(ypred > 0.5).sum()

68704

In [17]:
test.iloc[:,-1].sum()

74596

In [26]:
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, confusion_matrix
auc_lgbm = roc_auc_score(test.iloc[:,-1], ypred > 0.5)
acc_lgbm = accuracy_score(test.iloc[:,-1], ypred > 0.5)
rec_lgbm = recall_score(test.iloc[:,-1], ypred > 0.5)
f1_lgbm = f1_score(test.iloc[:,-1], ypred > 0.5)
con_lgbm = confusion_matrix(test.iloc[:,-1], ypred > 0.5)

In [27]:
print('auc:{0}'.format(auc_lgbm))
print('acc:{0}'.format(acc_lgbm))
print('rec:{0}'.format(rec_lgbm))
print('f1c:{0}'.format(f1_lgbm))
print('con:{0}'.format(con_lgbm))

auc:0.9503964684400282
acc:0.9987810120730333
rec:0.9010000536221782
f1c:0.9380460572226099
con:[[7207002    1493]
 [   7385   67211]]


In [21]:
acc_lgbm

0.9987810120730333

In [23]:
rec_lgbm

0.9010000536221782

In [ ]:
f1 = 

In [ ]:
ypred=lgbm.predict(test.iloc[:,:-1])
#predicting on test set
# ypred=lgbm.predict(x_test)

# #converting probabilities into 0 or 1
# for i in range(0,9769):
#     if ypred[i]>=.5:       # setting threshold to .5
#        ypred[i]=1
#     else:  
#        ypred[i]=0

#calculating accuracy
# accuracy_lgbm = accuracy_score(ypred,y_test)
# accuracy_lgbm




## 类别特征引入

### mean编码

### 用计数替换分类变量的每个标签

## drop columns

In [ ]:
# 3,4,5,6,15,16,17

## 超参数对比试验